<a href="https://colab.research.google.com/github/Snapchik/chat_analysis/blob/main/Chats_copmparisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import utils

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# For Colab usage
# hammas_chat = pd.read_json('/content/drive/My Drive/Colab Notebooks/result.json')

stopwords_english = stopwords.words('english')
stemmer = PorterStemmer()

hammas_chat = pd.read_json('Chats_exports/H.json')

# extracts date and text from a dictionary under similar key
hammas_norm = pd.json_normalize(hammas_chat['messages'])

hammas_norm_drop = hammas_norm.drop(columns=[
       'id', 'type','date_unixtime', 'actor', 'actor_id', 'action',
       'title', 'edited', 'edited_unixtime', 'from',
       'from_id', 'message_id', 'file', 'thumbnail', 'media_type', 'mime_type',
       'duration_seconds', 'width', 'height', 'photo', 'reply_to_message_id',
       'forwarded_from', 'performer', 'poll.question', 'poll.closed',
       'poll.total_voters', 'poll.answers','text_entities'
       ])
hammas_norm_drop = hammas_norm_drop.replace('', np.nan).dropna(subset = ['text'])
hammas_norm_drop['date']= pd.to_datetime(hammas_norm_drop['date'])

#extracting hashtags
hammas_norm_drop['hashtags'] = hammas_norm_drop['text'].apply(utils.extract_hashtags)

# apply the function to all rows
hammas_norm_drop['extracted_text'] = hammas_norm_drop.apply(utils.extract_text, axis=1)

#Some preprocessing steps that I found necessary for telegram chat

#clearing out some common misspellings and hyperlink
hammas_norm_drop['extracted_text'] = hammas_norm_drop['extracted_text'].str.replace('\n', '')
hammas_norm_drop['extracted_text'] = hammas_norm_drop['extracted_text'].str.replace('text_link', '')
hammas_norm_drop['extracted_text'] = hammas_norm_drop['extracted_text'].str.replace(r'https?://[^\s\n\r]+', '', regex=True)
hammas_norm_drop['extracted_text'] = hammas_norm_drop['extracted_text'].str.replace(r'bold\B', '', regex=True)
hammas_norm_drop['extracted_text'] = hammas_norm_drop['extracted_text'].str.replace(r'#', '',regex=True)
hammas_norm_drop['extracted_text'] = hammas_norm_drop['extracted_text'].str.replace('hashtag', '')
hammas_norm_drop['extracted_text'] = hammas_norm_drop['extracted_text'].str.replace('italic', '')
hammas_norm_drop['extracted_text'] = hammas_norm_drop['extracted_text'].str.replace('_', '')
hammas_norm_drop['extracted_text'] = hammas_norm_drop['extracted_text'].str.replace('—', '')


hammas_norm_drop['messages_token'] = hammas_norm_drop["extracted_text"].str.lower().apply(nltk.word_tokenize)


hammas_norm_drop['tokens_without_stopwords'] = hammas_norm_drop['messages_token'].apply(lambda x:
     [word for word in x if word not in (stopwords_english) and word not in string.punctuation])

hammas_norm_drop['stemmed_tokens_wt_stopwords_punctuation'] = hammas_norm_drop['tokens_without_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])

/Users/timurburhanov/Projects/GitHub/chat_analysis/utils.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in row[1]:


In [4]:
# For Colab usage
# IDF_chat = pd.read_json('/content/drive/My Drive/Colab Notebooks/IDF.json')


IDF_chat = pd.read_json('Chats_exports/IDF.json')

# extracts date and text from a dictionary under similar key
IDF_norm = pd.json_normalize(IDF_chat['messages'])

IDF_norm_drop = IDF_norm.drop(columns=['id', 'type','date_unixtime', 'actor', 'actor_id', 'action',
       'title', 'edited', 'edited_unixtime', 'from',
       'from_id', 'file', 'thumbnail', 'media_type', 'mime_type',
       'duration_seconds', 'width', 'height', 'photo', 'reply_to_message_id',
       'text_entities', 'duration', 'file_name'])
IDF_norm_drop = IDF_norm_drop.replace('', np.nan).dropna(subset = ['text'])
IDF_norm_drop['date']= pd.to_datetime(IDF_norm_drop['date'])

# apply the function to all rows
IDF_norm_drop['extracted_text'] = IDF_norm_drop.apply(utils.extract_text, axis=1)

#Big preprocessing steps

#clearing out some common misspellings and hyperlink
IDF_norm_drop['extracted_text'] = IDF_norm_drop['extracted_text'].str.replace('\n', '')
IDF_norm_drop['extracted_text'] = IDF_norm_drop['extracted_text'].str.replace('text_link', '')
IDF_norm_drop['extracted_text'] = IDF_norm_drop['extracted_text'].str.replace(r'https?://[^\s\n\r]+', '', regex=True)
IDF_norm_drop['extracted_text'] = IDF_norm_drop['extracted_text'].str.replace(r'bold\B', '', regex=True)
IDF_norm_drop['extracted_text'] = IDF_norm_drop['extracted_text'].str.replace(r'#', '',regex=True)
IDF_norm_drop['extracted_text'] = IDF_norm_drop['extracted_text'].str.replace('hashtag', '')
IDF_norm_drop['extracted_text'] = IDF_norm_drop['extracted_text'].str.replace('italic', '')
IDF_norm_drop['extracted_text'] = IDF_norm_drop['extracted_text'].str.replace('_', '')
IDF_norm_drop['extracted_text'] = IDF_norm_drop['extracted_text'].str.replace('—', '')
IDF_norm_drop['extracted_text'] = IDF_norm_drop['extracted_text'].str.replace('-', '')


#applying tokenizer
IDF_norm_drop['messages_token'] = IDF_norm_drop["extracted_text"].str.lower().apply(nltk.word_tokenize)


IDF_norm_drop['tokens_without_stopwords'] = IDF_norm_drop['messages_token'].apply(
     lambda x: [word for word in x if word not in stopwords_english and word not in string.punctuation])


IDF_norm_drop['stemmed_tokens_wt_stopwords_punctuation'] = IDF_norm_drop['tokens_without_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])

/Users/timurburhanov/Projects/GitHub/chat_analysis/utils.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in row[1]:


In [7]:
IDF_norm_drop['extracted_text'].to_pickle('idf_text')
hammas_norm_drop['extracted_text'].to_pickle('hammas_text')

IDF_norm_drop['stemmed_tokens_wt_stopwords_punctuation'].to_pickle('idf_tokens')
hammas_norm_drop['stemmed_tokens_wt_stopwords_punctuation'].to_pickle('hammas_tokens')